In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists Nuoma
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists Nuoma
    (
        miestas text,
        rajonas text,
        gatve text,
        kaina numeric,
        kv_kaina numeric
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [44]:
createMainTable('aruodas.db')


True

In [45]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butu-nuoma/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source

counts = 0
page = 1
while page < 128:
    source = driver.page_source
    
    bs = BeautifulSoup(source, 'html.parser')

    butai_nuorodos = []

    linkai = bs.find_all('h3')
    for linkas in linkai[1:]:
        l = linkas.find('a')
        if l != None:
            i = l['href']
                # print(i)
            butai_nuorodos.append(i)

    # kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
    # for k in kitas:
    #     # print(k.text.strip())
    #     if k.text.strip() == '»':
    #         kitas_psl = k['href']
    #         # print(kitas_psl)
    # next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'

    for nuoroda in butai_nuorodos:
        print(nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(2)
        source = driver.page_source

        cols_i_have = getColNamesList(table='Nuoma', dbname='aruodas.db')

        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None

        duomenys['nuoroda'] = nuoroda

        bs_buto = BeautifulSoup(source, 'html.parser')

        gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
        gyvenviete_lst = gyvenviete.split(',')
        if len(gyvenviete_lst) == 4:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            rajonas = gyvenviete_lst[1].strip()
            duomenys['rajonas'] = rajonas
            # print(rajonas)
            gatve = gyvenviete_lst[2].strip()
            duomenys['gatve'] = gatve
            # print(gatve)
        if len(gyvenviete_lst) == 3:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            gatve = gyvenviete_lst[1].strip()
            duomenys['gatve'] = gatve
            duomenys['rajonas'] = None

        try:    
            kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
            duomenys['kaina'] = int(kaina)
            # print(kaina)
        except:
            pass    

        try:
            kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
            # print(kv_kaina_txt)
            kv_kaina = float(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', '').replace(',', '.'))
            duomenys['kv_kaina'] = kv_kaina
            # print(kv_kaina)
        except:
            pass


        info_keys = []
        info_values = []
        info_k = bs_buto.find('dl', {'class':'obj-details'}).find_all('dt')
        for i in info_k:
            # print(i.text.strip())
            info_keys.append(i.text.strip())
            a = i.text.strip()
            # if a not in duomenys.keys():
            #     print(a)
        # print(info_keys)
        # print(len(info_keys))

        # info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('span',{'class':'fieldValueContainer'})
        info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('dd')
        for i in info_v:
            # print(i.text.strip())
            info_values.append(i.text.strip())
        # print(info_values)
        # print(len(info_values))

        for k, v in zip(info_keys, info_values):
            duomenys[k] = v

        # print(duomenys)
        cols_i_need = list(duomenys.keys())
        # print(cols_i_need)

        data = tuple(duomenys.values())

        cols_i_have = getColNamesList(table='Nuoma', dbname='aruodas.db')

        alterTableAddColumn(cols_i_need, cols_i_have, 'Nuoma', 'aruodas.db')

        writeMany(data, 'Nuoma', 'aruodas.db')
        counts = counts +1
        print(f'Įrašytas įrašas Nr {counts}, puslapis {page}')


    # driver.get(next_nuoroda)
    
    # su get metodu atsidarom next_nuoroda
    # nuskaitome next puslapi
    # pasiimam nuajas auto nuorodas ir nauja next nuorodas
    # cikla kartojam is naujo
    print(f'Įrašytas puslapis {page}')
    # page = page +1
    page = page +1
    next_nuoroda = f'https://www.aruodas.lt/butu-nuoma/puslapis/{page}/?FOwnerDbId0=1&FOwnerDbId2=1'
    driver.get(next_nuoroda)
    print(f'kitas puslapis: {page}')

driver.close()
print('Baigta!!!')

https://www.aruodas.lt/butu-nuoma-vilniuje-senamiestyje-t-vrublevskio-g-isnuomojamas-stilingai-irengtas-kambariu-4-1344235/
Įrašytas įrašas Nr 1, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-senamiestyje-a-vienuolio-g-laisva-nuo-gruodzio-rami-studija-miesto-4-1317808/
Įrašytas įrašas Nr 2, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-pasilaiciuose-perkunkiemio-g-butas-nuomojamas-be-tarpininku-agentu-4-669847/
Įrašytas įrašas Nr 3, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-snipiskese-olimpieciu-g-ypatingai-patogioje-vietoje-karaliaus-4-1304811/
Įrašytas įrašas Nr 4, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-snipiskese-zalgirio-g-isnuomojamas-k-butas-su-kv-m-balkonu-4-1349101/
Įrašytas įrašas Nr 5, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-naujininkuose-saltkalviu-g-salygos-gyvunai-nepageidaujami-4-1349543/
Įrašytas įrašas Nr 6, puslapis 1
https://www.aruodas.lt/butu-nuoma-vilniuje-fabijoniskese-p-zadeikos-g-isnuomojame-buta-kuris-tur

In [30]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butu-nuoma/puslapis/100/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [32]:
bs = BeautifulSoup(source, 'html.parser')

butai_nuorodos = []

linkai = bs.find_all('h3')
for linkas in linkai[1:]:
    l = linkas.find('a')
    if l != None:
        i = l['href']
        # if i != 'https://www.aruodas.lt/butai-vilniuje-pasilaiciuose-laisves-pr-4-27815396/':
        print(i)
        butai_nuorodos.append(i)

# linkai = bs.find_all('div', {'class':'list-adress-v2'})
# for linkas in linkai:
#     if linkas != None:
#         print('::REM::')
#         # if linkas != None:
#         link = linkas.find('h3')
#         print(link)
        
print(len(butai_nuorodos))

https://www.aruodas.lt/butu-nuoma-vilniuje-santariskese-daujoto-g-isnuomojamas-k-butas-naujosios-4-1352817/
https://www.aruodas.lt/butu-nuoma-vilniuje-naujamiestyje-k-donelaicio-g-elegantiskas-kvm-kambariu-butas-su-4-1352807/
https://www.aruodas.lt/butu-nuoma-vilniuje-paupyje-krivules-g-labai-patrauklioje-vietoje-nuomojamas-erdvus-4-1352805/
https://www.aruodas.lt/butu-nuoma-vilniuje-snipiskese-kalvariju-g-isnuomojami-apartamentai-snipiskese-4-1352803/
https://www.aruodas.lt/butu-nuoma-vilniuje-uzupyje-uzupio-g-isnuomojamas-kambariu-butas-uzupio-4-1352791/
https://www.aruodas.lt/butu-nuoma-vilniuje-naujininkuose-dzuku-g-nuomojamas-kambario-butas-naujininkuose-4-1352789/
https://www.aruodas.lt/butu-nuoma-vilniuje-justiniskese-sviliskiu-g-patogioje-vietoje-justiniskese-nuomuojamas-4-1352781/
https://www.aruodas.lt/butu-nuoma-vilniuje-naujamiestyje-raseiniu-g-prestizineje-vietoje-raseiniu-g-4-1352769/
https://www.aruodas.lt/butu-nuoma-vilniuje-naujojoje-vilnioje-darzelio-g-vilnius-naujoji

In [34]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butu-nuoma-vilniuje-senamiestyje-t-vrublevskio-g-isnuomojamas-stilingai-irengtas-kambariu-4-1344235/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [42]:
bs_buto = BeautifulSoup(source, 'html.parser')

duomenys = {}
gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
gyvenviete_lst = gyvenviete.split(',')
if len(gyvenviete_lst) == 4:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    # print(miestas)
    rajonas = gyvenviete_lst[1].strip()
    duomenys['rajonas'] = rajonas
    # print(rajonas)
    gatve = gyvenviete_lst[2].strip()
    duomenys['gatve'] = gatve
    print(gatve)
if len(gyvenviete_lst) == 3:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    # print(miestas)
    gatve = gyvenviete_lst[1].strip()
    duomenys['gatve'] = gatve
    duomenys['rajonas'] = None

try:    
    kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
    duomenys['kaina'] = kaina
    print(kaina)
except:
    pass    


kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
print(kv_kaina_txt)
kv_kaina = float(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', '').replace(',', '.'))
# duomenys['kv_kaina'] = kv_kaina
print(kv_kaina)



T. Vrublevskio g.
2200
(14,10 €/m²)
14.1
